In [16]:
%pwd

'd:\\Advanced Project\\Lesion-Stroke-Segmentation'

In [14]:
import os
os.chdir("../")

In [17]:
%pwd

'd:\\Advanced Project\\Lesion-Stroke-Segmentation'

# Cutome Logger

In [58]:
import os
import sys
import logging

In [59]:
logging_format = "[%(asctime)s]: %(levelname)s: %(module)s : %(message)s"
log_dir = "logs"
log_filepath = os.path.join(log_dir,"running_logs.log")
os.makedirs(log_dir, exist_ok=True)

In [60]:
logging.basicConfig(
    format=logging_format,
    level=logging.INFO,

    handlers=[
        logging.FileHandler(filename=log_filepath),
        logging.StreamHandler(sys.stdout)
    ]
)

In [61]:
logger = logging.getLogger("lesionSegmentationlogger")

# Custom Exception

In [4]:
import sys

def error_message_detail(error, error_detail: sys):
    _,_,exc_tb = error_detail.exc_info()
    file_name = exc_tb.tb_frame.f_code.co_filename
    line_number = exc_tb.tb_lineno


    error_message = f"File Name: {file_name}\nLine Number: {line_number}\nError: {error}"

    return error_message

In [5]:
class CustomeException(Exception):
    def __init__(self, error_message, error_detail:sys):
        super().__init__(error_message)
        self.error_message = error_message_detail(error=error_message, error_detail=error_detail)

    def __str__(self):
        return self.error_message

In [8]:
try:
    a = 9/0
    print(a)
except Exception as e:
    raise CustomeException(e, sys)

CustomeException: File Name: C:\Users\ehtes\AppData\Local\Temp\ipykernel_2460\3557617741.py
Line Number: 2
Error: division by zero

# Read YAML Files

In [49]:
import yaml
from pathlib import Path
from box import ConfigBox
from box.exceptions import BoxValueError
from ensure import ensure_annotations, EnsureError

In [45]:
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

In [50]:
def catch_ensure_errors(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except EnsureError as e:
            raise CustomeException(e,sys)
    return wrapper

In [62]:
@catch_ensure_errors
@ensure_annotations
def read_yaml(file_path: Path) -> ConfigBox:
    try:
        with open(file=file_path) as yaml_file:
            data = yaml.safe_load(yaml_file)
            logger.info(f"Read YAML File: {file_path}")
            return ConfigBox(data)
    except BoxValueError:
        raise ValueError("YAML FIle is EMPTY")
    except Exception as e:
        raise CustomeException(e, sys)

In [63]:
data = read_yaml(CONFIG_FILE_PATH)

[2025-02-03 15:37:34,163]: INFO: 1881749221 : Read YAML File: config\config.yaml


# Make Directories (Creating Folders Only)

In [53]:
import os
from pathlib import Path

In [67]:
@catch_ensure_errors
@ensure_annotations
def create_directory(lst_dir:list):
    for dir in lst_dir:
        dir = Path(dir)
        os.makedirs(dir, exist_ok=True)
        logger.info(f"Directory has been Created: {dir}")

In [68]:
create_directory(["artifacts", "artifacts/data_ingestion"])

[2025-02-03 15:40:00,707]: INFO: 176530609 : Directory has been Created: artifacts
[2025-02-03 15:40:00,712]: INFO: 176530609 : Directory has been Created: artifacts\data_ingestion
